# Setup Jogo

## Instalar Gym-Retro e Bibliotecas Necessarias

In [ ]:
#!pip install gym==0.19.0 gym-retro 

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
# No prompt da anaconda ative o venv que ira utilizar, exemplo: "conda activate envtest". Apos ativar entre na sua pasta roms (com a ROM do jogo dentro dessa pasta) e execute o comando abaixo.
# python -m retro.import .

In [ ]:
#!pip install opencv-python

In [ ]:
#!pip install ipywidgets 

In [ ]:
#!pip install stable-baselines3[extra]==1.3.0 optuna --user

In [1]:
#python < 3.8 
#Verifique que o python esta entre a versao 3.7 e 3.8 pois o gym retro so funciona nessas versoes
import sys
sys.version


'3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]'

## Importar Dependencias

In [2]:
#importando retro para usar MK ROM
import retro
# Sera usado para diminuir velocidade do jogo
import time

import os

In [4]:
#Se erro do tipo ja existe um env aberto rodar esse codigo
env.close()

In [5]:
#Inicia jogo
env = retro.make(game='MortalKombatII-Genesis')
#env.reset()


In [6]:
# Verificando se o observation space esta funcionando
env.observation_space

Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
 

In [7]:
# Verificando se o action space esta funcionando
env.action_space.sample()

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1], dtype=int8)

# Setup env
Vamos otimizar removendo as cores (agente nao precisa ver cores) e diminuir observation space do agente

In [8]:
from gym import Env 
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [11]:
#Custom env, baseado em: https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html
class MortalKombat(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Env.observation_space
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype = np.uint8) #O que o agente "ve"
        self.action_space = MultiBinary(12) # O que o agente pode fazer (acoes)
        #self.game = retro.make(game='MortalKombatII-Genesis', obs_type='image', use_restricted_actions=retro.Actions.FILTERED) # retro.Actions.FILTERED nos da somente acoes validas
        self.game = retro.make(game='MortalKombatII-Genesis', use_restricted_actions=retro.Actions.FILTERED ) # retro.Actions.FILTERED nos da somente acoes validas
        #env = retro.make(gamename, state='firstfightveryeasy.state', obs_type=retro.Observations.IMAGE)
        #self.game.reset()
    def reset(self):
        #Recomeca jogo
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame=obs
        
        # Infos (guardar o score a cada frame)
        self.score = 0
        return obs
    
    def render(self, *args, **kwargs):
        self.game.render()
    
    def close(self):
        self.game.close()
        
    
    #Vamos transformar o jogo em cinza e diminuir o tamanho 
    def preprocess(self, observation):
        #Transforma em cinza
        cinza = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        #Resize
        resize = cv2.resize(cinza, (84,84), interpolation = cv2.INTER_CUBIC)
        final = np.reshape(resize, (84,84,1))
        return final
        
    def step(self, action):
        obs, self.reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) #Otimizacoes comentadas acimas...
        
        frame_change = obs - self.previous_frame #Pixels que mudaram do ultimo frame para o novo
        self.previous_frame = obs
        
        #Reward (depois mudar os parametros)
        self.reward = info['rounds_won']*500 + info['enemy_rounds_won']*-500 + info['health']*1 + info['enemy_health']*-1 - self.score
        
        
        self.score = info['rounds_won']*500 + info['enemy_rounds_won']*-500 + info['health'] + info['enemy_health']*-1
    
        return frame_change, self.reward, done, info
        
    

In [12]:
env.close()
env = MortalKombat()
#env.reset()
    

In [ ]:
# IA faz movimento aleatorios nesse teste
obs = env.reset()
dead = False
for game in range(1): 
    while not dead: 
        if dead: 
            obs = env.reset()
        env.render()
        obs, reward, dead, info = env.step(env.action_space.sample())
        time.sleep(0.01) 
        print(reward)

# Hiperparametros & Optuna

In [13]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponível!")
    print("Versão do CUDA:", torch.version.cuda)
else:
    print("CUDA não está disponível.")



CUDA está disponível!
Versão do CUDA: 11.7


In [14]:
import stable_baselines3

print(stable_baselines3.__version__)


1.3.0


In [58]:
# Metodos e bibliotecas que utilizaremos
import optuna
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecTransposeImage
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import VecVideoRecorder
from gym.wrappers import FrameStack
from gym.wrappers import FrameStack


In [16]:
LOG_D = './logs/'
OPT_D= './opt/'
# Vamos salvar o resultados do Optuna nesse path
PATH = os.path.join(OPT_D, 'v_{}_best_model'.format(1)) 

In [17]:
PATH

'./opt/v_1_best_model'

In [18]:
#Usado pelo optuna para sugerir um valor para os hiperparametros
#Valores sugeridos 
def opt_objective(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_float('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_float('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_float('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_float('gae_lambda', 0.8, 0.99)
    }

In [19]:
def optimize_agent(trial):
    try:
        parametros = opt_objective(trial) # Pegamos valores para os hiperparametros da funcao testamos

        env = MortalKombat()
        #Monitora o env para depois podermos visualizar como as configuracoes de hiperparametros se sairam
        env = Monitor(env, LOG_D)
        env = DummyVecEnv([lambda: env])
        #env = CustomDummyVecEnv([lambda: env])
        # Nosso MDPs tera 4 frames de "memoria" para tomar a proxima decisao
        env = VecFrameStack(env, n_stack=4, channels_order='last')
        #env = FrameStack(env, 4)
        #env = VecTransposeImage(env)

        ppo_mod = PPO('CnnPolicy', env, tensorboard_log=LOG_D, verbose=0, **parametros) #utilizamos o PPO para resolver o MDPs
        ppo_mod.learn(total_timesteps=100000)
        #model.learn(total_timesteps=100)

        mean_reward, _ = evaluate_policy(ppo_mod, env, n_eval_episodes=5) #n_eval_episodes é em quantos jogos essa configuracao sera testada
        env.close()

        PATH = os.path.join(OPT_D, 'v_{}_best_model'.format(trial.number))
        ppo_mod.save(PATH)

        return mean_reward 

    except Exception as e: #Algumas configuracoes de hiperparametros podem nao funcionar por isso a try
        return -100000

In [20]:
env.close() #Para fechar um env caso esteja aberto

In [ ]:
#Realizando o estudo de hiperparametros
study = optuna.create_study(direction='maximize') #Por default o optuna tenta minimizar a funcao
#study.optimize(optimize_agent, n_trials=100, n_jobs=1) 
study.optimize(optimize_agent, n_trials=5, n_jobs=1)

In [ ]:
number_x = study.best_trial.number
print(number_x)
path_v_best_model = 'v_{}_best_model.zip'.format(number_x)
#path_v_best_model = 'v_{}_best_model.zip'.format('-1')
best_model = PPO.load(os.path.join(OPT_D, path_v_best_model))
print(path_v_best_model)

# Treinamento

In [21]:
# Classe padrao baseada no callback da documentacao do stable baselines: https://stable-baselines3.readthedocs.io/en/master/guide/examples.html#basic-usage-training-saving-loading
class CustomCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(CustomCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            best_model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(best_model_path)

        return True

In [22]:
TRAIN_D = './train/'

In [23]:
callback = CustomCallback(check_freq=10000, save_path=TRAIN_D)

In [24]:
env.close()

In [56]:

env = MortalKombat()
env = Monitor(env, LOG_D)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)



In [ ]:
best_model_param = study.best_params
best_model_param['n_steps'] = ((best_model_param['n_steps'] + 63) // 64) * 64 #tornando o n_steps divisivel por 64
#Pode ser necessario diminuir a lrate caso nao esteja convergindo adequadamente
best_model_param['learning_rate']= 1.182030842374264e-06
best_model_param


In [31]:
#Caso queira definir valores manualmente
best_model_param = {'n_steps': 7360,
 'gamma': 0.8403833640495785,
 'learning_rate': 4.04419273011491e-05,
 'clip_range': 0.27718291020022345,
 'gae_lambda': 0.8009321244173684}    
best_model_param

{'n_steps': 7360,
 'gamma': 0.8403833640495785,
 'learning_rate': 4.04419273011491e-05,
 'clip_range': 0.27718291020022345,
 'gae_lambda': 0.8009321244173684}

In [40]:
ppo_model = PPO('CnnPolicy', env, tensorboard_log=LOG_D, verbose=1, **best_model_param,ent_coef=0.01) #utilizamos o PPO para resolver o MDPs


Using cuda device
Wrapping the env in a VecTransposeImage.


In [41]:
#ppo_model.load(os.path.join(OPT_D, path_v_best_model))
ppo_model.load(os.path.join('./backup/backup1.zip'))
#ppo_model.load(os.path.join(TRAIN_D, 'best_model_1000000.zip'))
#print(path_v_best_model)


In [43]:
ppo_model.learn(total_timesteps=1100000, callback=callback)

Logging to ./logs/PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.94e+03 |
|    ep_rew_mean     | -835     |
| time/              |          |
|    fps             | 386      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 7360     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.44e+03   |
|    ep_rew_mean          | -982       |
| time/                   |            |
|    fps                  | 295        |
|    iterations           | 2          |
|    time_elapsed         | 49         |
|    total_timesteps      | 14720      |
| train/                  |            |
|    approx_kl            | 0.01953918 |
|    clip_fraction        | 0.119      |
|    clip_range           | 0.277      |
|    entropy_loss         | -8.07      |
|    explained_variance   | 0.0171     |
|    learning_rate   

KeyboardInterrupt: 

# Avaliar Modelos Treinados

In [49]:
#path_train = './opt/v_-1_best_model.zip'
#path_train = './train/best_model_260000.zip'
path_train = './backup/backup1.zip'
ppo_model = PPO.load(path_train, env=env)

Wrapping the env in a VecTransposeImage.


In [45]:
mean_score, _ = evaluate_policy(ppo_model, env, n_eval_episodes=2, render=True, deterministic=True)
print (mean_score)

-1120.0


In [53]:
env.close()

# Testes

In [57]:
obs = env.reset()
dead = False
for game in range(1): 
    while not dead: 
        if dead: 
            obs = env.reset()
        env.render()
        predict = ppo_model.predict(obs)[0]
        obs, reward, dead, info = env.step(predict)
        time.sleep(0.001)
        total_reward+=reward
        cont+=1
        if reward>0:
            print(reward)
            
#print(total_reward)


[20.]
[20.]
[80.]
[20.]
[24.]
[20.]
[20.]
[20.]
[16.]
[500.]
[20.]
[20.]
[15.]
[20.]


In [ ]:
ppo_model.save('./backup/')